In [ ]:
import pandas as pd
pd.set_option('max_colwidth', 80)

### Merging multiple csvs

In [ ]:
import glob
df = pd.DataFrame()
for file in glob.glob("*ig.csv"):
    brand = file.split("_ig.csv")[0]
    temp_df = pd.read_csv(file)
    if brand == "samsung":
        brand = "samsungmobile"
    temp_df["brand"] = brand
    df = pd.concat([df,temp_df], axis=0)

In [ ]:
def handle_numbers(string):
    string = str(string).replace(",","")
    if "k" in string:
        if "." in string:
            return int(string.replace("k","00").replace(".",""))
        else:
            return int(string.replace("k","00"))
    if "m" in string:
        if "." in string:
            return int(string.replace("m","00000").replace(".",""))
        else:
            return int(string.replace("m","000000"))
    return int(float(string))
        

In [ ]:
# only choose images
df = df[df.media=="image"]

In [ ]:
df.likes = df.likes.apply(handle_numbers)

In [ ]:
df = df[df.caption.notnull()]

### Feature Engineering

In [ ]:
from sklearn.pipeline import make_union
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline

In [ ]:
def get_text(df):
    return df.caption

In [ ]:
get_text_ft = FunctionTransformer(get_text, validate=False)

Get page meta data

In [ ]:
meta_df = pd.read_csv("brand_ig_data.csv")

In [ ]:
meta_df = meta_df.loc[:, ["brand_ig","n_posts", "n_followers","n_following"]]

In [ ]:
df.rename(index=str, columns={"brand": "brand_ig"}, inplace=True)

In [ ]:
final_df = pd.merge(df,meta_df,how="left",on="brand_ig")

In [ ]:
final_df["n_posts"] = final_df.n_posts.apply(handle_numbers)

In [ ]:
final_df["n_followers"] = final_df.n_followers.apply(handle_numbers)

In [ ]:
# Remove posts with zero likes (to take a log transform and calculate MAPE)
final_df = final_df[final_df.likes != 0]

In [ ]:
def get_brand_meta(df):
    return df.loc[:,["n_posts", "n_followers"]]

In [ ]:
get_brand_meta_ft = FunctionTransformer(get_brand_meta, validate=False)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(stop_words='english', max_df=0.3, min_df=4)

In [ ]:
union = make_union(make_pipeline(get_text_ft, vect), get_brand_meta_ft)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(final_df, final_df.likes, random_state=1)

In [ ]:
X_train_meta_dtm = union.fit_transform(X_train)

In [ ]:
X_test_meta_dtm = union.transform(X_test)

### Model Selection

In [ ]:
import sklearn.linear_model
import sklearn.datasets
import sklearn.svm
from sklearn.metrics import r2_score
import sklearn.feature_extraction.text
import sklearn.utils.sparsefuncs

In [ ]:
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred)/ y_true)) * 100

Standard LR

In [ ]:
regression = sklearn.linear_model.LinearRegression()
regression.fit(X_train_meta_dtm, np.log(y_train))

In [ ]:
yhat_lr = regression.predict(X_test_meta_dtm)

In [ ]:
print("r2: %.3f"%(r2_score(np.log(y_test),yhat_lr)))
print("mape: %.3f"%mean_absolute_percentage_error(np.log(y_test),yhat_lr))

lasso, L1 penalized regression

In [ ]:
lasso = sklearn.linear_model.Lasso(alpha = 0.1, max_iter=3000)

In [ ]:
lasso.fit(X_train_meta_dtm,np.log(y_train))

In [ ]:
yhat_lasso = lasso.predict(X_test_meta_dtm)

In [ ]:
print("r2: %.3f"%(r2_score(np.log(y_test),yhat_lasso)))
print("mape: %.3f"%mean_absolute_percentage_error(np.log(y_test),yhat_lasso))

Adaboost

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
ada = AdaBoostRegressor(base_estimator=None, n_estimators=200, learning_rate=1, loss="linear", random_state=1)

In [ ]:
ada.fit(X_train_meta_dtm, np.log(y_train))

In [ ]:
yhat_ada = ada.predict(X_test_meta_dtm)

In [ ]:
print("r2: %.3f"%(r2_score(np.log(y_test),yhat_ada)))
print("mape: %.3f"%mean_absolute_percentage_error(np.log(y_test),yhat_ada))

Gradient Boosted Trees

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
gbr = GradientBoostingRegressor(loss="ls", learning_rate=0.1, n_estimators=100, subsample=1.0, criterion="friedman_mse", min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, init=None, random_state=1, max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, warm_start=False, presort="auto")

In [ ]:
gbr.fit(X_train_meta_dtm, np.log(y_train))

In [ ]:
yhat_gbr = gbr.predict(X_test_meta_dtm)

In [ ]:
print("r2: %.3f"%(r2_score(np.log(y_test),yhat_gbr)))
print("mape: %.3f"%mean_absolute_percentage_error(np.log(y_test),yhat_gbr))

### Model Interpretation

In [ ]:
threshold = 1.0e-03

In [ ]:
# Ignore the long tail
gbr.feature_importances_[gbr.feature_importances_ >= threshold].sum()

In [ ]:
impt_feats = gbr.feature_importances_ >= threshold

In [ ]:
impt_ind = np.where(impt_feats[:-2])[0]

In [ ]:
np.array(vect.get_feature_names())[impt_feats[:-2]]

In [ ]:
vect.fit(X_train.caption)

In [ ]:
gbr.feature_importances_[impt_feats].size

In [ ]:
importance_df = pd.DataFrame()

In [ ]:
importance_df["word"] = np.array(vect.get_feature_names())[impt_feats[:-2]]

In [ ]:
importance_df["importance"] = gbr.feature_importances_[impt_feats][:-2]

In [ ]:
importance_df = importance_df.sort_values(by="importance",ascending=False)

In [ ]:
meta_importance_df = pd.DataFrame([["n_followers",0.24085896],["n_posts",0.08033263,]],columns=["word","importance"])

In [ ]:
pd.concat([meta_importance_df,importance_df],axis = 0).reset_index(drop=True).head(20)

### Select only useful features and use all data in final fitting

In [ ]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
# model = SelectFromModel(gbr, prefit=True)
# X_train_new = model.transform(X_train_meta_dtm)
# X_test_new = model.transform(X_test_meta_dtm)

In [ ]:
X_final = np.vstack([X_train_new.toarray(),X_test_new.toarray()])
y_final = np.concatenate([y_train.values, y_test.values])

In [ ]:
gbr.fit(X_final, np.log(y_final))

In [ ]:
yhat_gbr_2 = gbr.predict(X_test_new)

In [ ]:
print("r2: %.3f"%(r2_score(np.log(y_test),yhat_gbr_2)))
print("mape: %.3f"%mean_absolute_percentage_error(np.log(y_test),yhat_gbr_2))

### Creating a new entry

In [ ]:
final_df.iloc[600]

In [ ]:
 test_caption = final_df.iloc[600].caption

In [ ]:
def construct_test_df(test_caption, brand_posts, brand_followers):
    test_df = pd.DataFrame([[test_caption,brand_posts,brand_followers]],columns=["caption","n_posts","n_followers"])
    return test_df

In [ ]:
construct_test_df(test_caption,152,4700000)

In [ ]:
test_caption = 'Moments are the best souvenirs you can collect. #GalaxyS9plus 📷: @inescostamonteiro'

In [ ]:
test1 = union.transform(construct_test_df(test_caption,274,4200000))

In [ ]:
gbr.predict(test1)



### Pickle file for later use

In [1]:
import pickle

In [ ]:
with open("instagram_model.pkl","wb") as f:
    pickle.dump(gbr, f)

### Unpickling

In [ ]:
with open("instagram_model.pkl","rb") as f:
    gbr_insta = pickle.load(f)

In [ ]:
gbr_insta.predict(X_test_new)

### Facebook Data

In [ ]:
fb_df = pd.read_csv("fb_final.csv")

In [ ]:
fb_df.page_followers = fb_df.page_followers.apply(lambda x: int(x.replace(",","")))

In [ ]:
fb_df.page_likes = fb_df.page_likes.apply(lambda x: int(x.replace(",","")))

In [ ]:
#only dropping 2 rows
fb_df = fb_df[fb_df.post_likes != 0]

In [ ]:
fb_df

In [ ]:
def get_text_fb(df):
    return df.post_message

In [ ]:
get_text_fb_ft = FunctionTransformer(get_text_fb, validate=False)

Get page meta data

In [ ]:
def get_brand_meta_fb(df):
    return df.loc[:,["page_likes", "page_followers"]]

In [ ]:
get_brand_meta_fb_ft = FunctionTransformer(get_brand_meta_fb, validate=False)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect_fb = CountVectorizer(stop_words='english', max_df=0.3, min_df=4)

In [ ]:
union_fb = make_union(make_pipeline(get_text_fb_ft, vect_fb), get_brand_meta_fb_ft)

In [ ]:
from sklearn.model_selection import train_test_split
X_train_fb, X_test_fb, y_train_fb, y_test_fb = train_test_split(fb_df, fb_df.post_likes, random_state=1)

In [ ]:
X_train_meta_dtm_fb = union_fb.fit_transform(X_train_fb)

In [ ]:
X_test_meta_dtm_fb = union_fb.transform(X_test_fb)

In [ ]:
gbr_fb = GradientBoostingRegressor(loss="ls", learning_rate=0.1, n_estimators=100, subsample=1.0, criterion="friedman_mse", min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, init=None, random_state=1, max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, warm_start=False, presort="auto")

In [ ]:
gbr_fb.fit(X_train_meta_dtm_fb, y_train_fb)

In [ ]:
yhat_gbr_fb = gbr_fb.predict(X_test_meta_dtm_fb)

In [ ]:
print("r2: %.3f"%(r2_score(y_test_fb,yhat_gbr_fb)))
print("mape: %.3f"%mean_absolute_percentage_error(y_test_fb,yhat_gbr_fb))

In [ ]:
fb_df.post_likes.var()

In [ ]:
final_df.likes.var()

In [ ]:
fb_threshold = 1.0e-03

In [ ]:
# Ignore the long tail
gbr_fb.feature_importances_[gbr_fb.feature_importances_ >= fb_threshold].sum()

In [ ]:
impt_feats_fb = gbr_fb.feature_importances_ >= threshold

In [ ]:
impt_ind_fb = np.where(impt_feats_fb[:-2])[0]

In [ ]:
np.array(vect_fb.get_feature_names())[impt_feats_fb[:-2]]

In [ ]:
vect_fb.fit(X_train_fb.post_message)

In [ ]:
gbr_fb.feature_importances_[impt_feats_fb].size

In [ ]:
gbr_fb.feature_importances_

In [ ]:
importance_df_fb = pd.DataFrame()

In [ ]:
importance_df_fb["word"] = np.array(vect_fb.get_feature_names())[impt_feats_fb[:-2]]

In [ ]:
importance_df_fb["importance"] = gbr_fb.feature_importances_[impt_feats_fb][:-2]

In [ ]:
importance_df_fb = importance_df_fb.sort_values(by="importance",ascending=False)

In [ ]:
meta_importance_df_fb = pd.DataFrame([["page_likes",0.00352126],["page_followers",0.01468663]],columns=["word","importance"])

In [ ]:
pd.concat([meta_importance_df_fb,importance_df_fb],axis = 0).reset_index(drop=True).head(20)

In [ ]:
pd.concat([meta_importance_df_fb,importance_df_fb],axis = 0).reset_index(drop=True).head(20)

In [ ]:
with open("facebook_model.pkl","wb") as f:
    pickle.dump(gbr_fb, f)